# Titanic: Predict suvival on the Titanic with AWS Glue.
### 타이타닉 사고에서 어떤 승객이 살아남을 수 있을 지 예측해보세요
##### Kaggle은 2010년 설립된 예측모델 및 분석 대회 플랫폼입니다.
##### 기업 및 단체에서 데이터와 해결과제를 등록하면, 데이터 과학자들이 이를 해결하는 모델을 개발하고 경쟁하는 곳입니다. 
##### 이번 Lab에서는 Kaggle에서 입문자용 tutorial로 사용되는 titanic competition을 Glue와 Spark ML을 사용해서 데이터 분석, ETL, ML을 사용한 Prediction까지 실습해보도록 하겠습니다.

![titanic_sinking](images/titanic_sinking.jpg)

#### 필요한 라이브러리 Import

In [197]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean, col, split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

### 1. Glue 카탈로그에서 필요한 데이터 준비

In [198]:
from awsglue.context import GlueContext
 
glueContext = GlueContext(sc)
titanic_df = glueContext.create_dynamic_frame.from_catalog(database='analytics_hol',
                                                           table_name='titanic_train',                           
                                                           transformation_ctx='titanic_df').toDF()
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|passengerid|survived|pclass|                name|   sex| age|sibsp|parch|          ticket|   fare|cabin|embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|     Mr. Owen Harris|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|
|          2|       1|     1|Mrs. John Bradley...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|         Miss. Laina|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|
|          4|       1|     1|Mrs. Jacques Heat...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|   Mr. William Henry|  male|35.0|    0|    0|          373450|   8.05|     |       S|
|          6|       0|     3|           Mr. James|  male|null|    0|    0|      

##### passengerid: 승객 id
##### survived: 생존여부 (0=No, 1=Yes)
##### pclass: 티켓등급 (1=1st, 2=2nd, 3=3rd)
##### name: 이름
##### sex: 성별 (0=male, 1=female)
##### age: 나이
##### sibsp: 함께 탑승한 형제자매, 배우자의 수
##### parch: 함께 탑승한 부모, 자식의 수
##### ticket: 티켓번호
##### fare: 운임
##### cabin: 객실 번호
##### embarked: 탑승항구 (C=Cherbourg, Q=Queenstown, S=Southampton)

#### 1.1. 전체 데이터 수 확인

In [199]:
titanic_df.count()

891

#### 1.2. 데이터 스키마 확인

In [200]:
titanic_df.printSchema()

root
 |-- passengerid: long (nullable = true)
 |-- survived: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: long (nullable = true)
 |-- parch: long (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)

#### 1.3. 사용할 데이터에 모든 컬럼에 대해 요약 통계를 확인합니다.

In [201]:
titanic_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      passengerid|           survived|            pclass|                name|   sex|               age|             sibsp|              parch|            ticket|             fare|cabin|embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  891|     891|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

##### 전체 데이터가 891개로 다른 모든 컬럼은 891개의 데이터를 모두 가지고 있지만 age만 714개로 누락된 데이터가 존재함을 확인합니다.
##### 빈 스트링은 이 요약 통계에서 구분이 불가능함으로 이후에 별도의 확인 과정을 통해 따로 체크하도록 합니다.

### 2. Exploratory data analysis

#### 2.1. 탑승객 생존율 확인

In [202]:
titanic_df.groupBy('survived').count().show()

+--------+-----+
|survived|count|
+--------+-----+
|       0|  549|
|       1|  342|
+--------+-----+

##### 891명의 승객 중 342명만 생존했음을 알 수 있습니다.
##### 생존율은 sex, age, pclass와 같은 승객 특징들과 관련이 있으므로 해당 feature에 따른 생존율을 살펴봅니다.

#### 2.2. 성별에 따른 생존율 확인

In [203]:
titanic_df.groupBy('sex', 'survived').count().orderBy('sex', 'survived', ascending=[1, 1]).show()

+------+--------+-----+
|   sex|survived|count|
+------+--------+-----+
|female|       0|   81|
|female|       1|  233|
|  male|       0|  468|
|  male|       1|  109|
+------+--------+-----+

##### 남성보다 여성의 생존율이 높음을 알 수 있습니다.

#### 2.3. 티켓등급에 따른 생존율 확인

In [204]:
titanic_df.groupBy('pclass', 'survived').count().orderBy('pclass', 'survived', ascending=[1, 1]).show()

+------+--------+-----+
|pclass|survived|count|
+------+--------+-----+
|     1|       0|   80|
|     1|       1|  136|
|     2|       0|   97|
|     2|       1|   87|
|     3|       0|  372|
|     3|       1|  119|
+------+--------+-----+

##### 1등석의 생존율은 높고, 3등석의 생존율은 낮은 것을 알 수 있습니다.

#### 2.4. 함께 탑승한 형제자매나 배우자에 따른 생존율 확인

In [205]:
titanic_df.groupBy('sibsp', 'survived').count().orderBy('sibsp', 'survived', ascending=[1, 1]).show()

+-----+--------+-----+
|sibsp|survived|count|
+-----+--------+-----+
|    0|       0|  398|
|    0|       1|  210|
|    1|       0|   97|
|    1|       1|  112|
|    2|       0|   15|
|    2|       1|   13|
|    3|       0|   12|
|    3|       1|    4|
|    4|       0|   15|
|    4|       1|    3|
|    5|       0|    5|
|    8|       0|    7|
+-----+--------+-----+

##### 동승한 형제나매나 배우자가 없는 경우 생존율이 낮은 것을 알 수 있다.

#### 2.5. 함께 탑승한 부모님이나 자녀에 따른 생존율 확인

In [206]:
titanic_df.groupBy('parch', 'survived').count().orderBy('parch', 'survived', ascending=[1, 1]).show()

+-----+--------+-----+
|parch|survived|count|
+-----+--------+-----+
|    0|       0|  445|
|    0|       1|  233|
|    1|       0|   53|
|    1|       1|   65|
|    2|       0|   40|
|    2|       1|   40|
|    3|       0|    2|
|    3|       1|    3|
|    4|       0|    4|
|    5|       0|    4|
|    5|       1|    1|
|    6|       0|    1|
+-----+--------+-----+

##### 동승한 부모가 없는 경우 생존율이 낮은 것을 알 수 있다.

#### 2.6 탑승항구에 따른 생존율 확인

In [207]:
titanic_df.groupBy('embarked', 'survived').count().orderBy('embarked', 'survived', ascending=[1, 1]).show()

+--------+--------+-----+
|embarked|survived|count|
+--------+--------+-----+
|        |       1|    2|
|       C|       0|   75|
|       C|       1|   93|
|       Q|       0|   47|
|       Q|       1|   30|
|       S|       0|  427|
|       S|       1|  217|
+--------+--------+-----+

##### 2개의 데이터는 탑승항구가 명시되어 있지 않는 것을 알 수 있습니다.
##### 탑승항구 S(Southampton)의 생존율이 낮은 것을 알 수 있습니다.
##### 따라서, 탑승항구 S(Southampton)의 생존율이 낮은 이유를 알기 위해 탑승항구와 탑승석의 관계를 살펴볼 필요가 있습니다.

In [208]:
titanic_df.groupBy('embarked', 'pclass').count().orderBy('embarked', 'pclass', ascending=[1, 1]).show()

+--------+------+-----+
|embarked|pclass|count|
+--------+------+-----+
|        |     1|    2|
|       C|     1|   85|
|       C|     2|   17|
|       C|     3|   66|
|       Q|     1|    2|
|       Q|     2|    3|
|       Q|     3|   72|
|       S|     1|  127|
|       S|     2|  164|
|       S|     3|  353|
+--------+------+-----+

##### 탑승항구 S(Southampton)의 생존율이 낮은 이유는 대다수 탑승객이 3등석이기 때문임을 알 수 있습니다.

### 3. Feature engineering

#### 3.1. Null Column Check
##### Null 값이나 empty string을 가진 Column이 존재하는 지 확인합니다.

##### 3.1.1. Null Value를 체크하기 위한 UDF 함수 선언

In [209]:
def null_value_count(df):
    null_columns_counts = []
    
    for column in df.columns:
        nullRows = df.where((col(column).isNull()) | (col(column) == '')).count()
    
        if(nullRows > 0):
              null_columns_counts.append((column, nullRows))
    return(null_columns_counts)

##### 3.1.2. null_value_count UDF를 이용하여 Null Value feature 확인

In [210]:
null_columns_count_list = null_value_count(titanic_df)
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   age|              177|
|                 cabin|              687|
|              embarked|                2|
+----------------------+-----------------+

##### age feature가 177개의 null value를 가진 것이 확인되었습니다.
##### cabin feature가 687개의 empty string을 가진 것이 확인되었습니다.
##### embarked feature가 2개의 empty string을 가진 것이 확인되었습니다.
##### 이 정보는 이후 진행되는 feature engineering에서 참고 자료로 사용합니다.

#### 3.2. Name

In [211]:
titanic_df.select("name").show()

+--------------------+
|                name|
+--------------------+
|     Mr. Owen Harris|
|Mrs. John Bradley...|
|         Miss. Laina|
|Mrs. Jacques Heat...|
|   Mr. William Henry|
|           Mr. James|
|       Mr. Timothy J|
|Master. Gosta Leo...|
|Mrs. Oscar W (Eli...|
|Mrs. Nicholas (Ad...|
|Miss. Marguerite Rut|
|     Miss. Elizabeth|
|   Mr. William Henry|
|    Mr. Anders Johan|
|Miss. Hulda Amand...|
|Mrs. (Mary D King...|
|      Master. Eugene|
|  Mr. Charles Eugene|
|Vander Mrs. Juliu...|
|         Mrs. Fatima|
+--------------------+
only showing top 20 rows

##### 샘플 데이터에서 name field를 살펴보면 그대로 사용하기는 어렵고 정리가 필요하다는 것을 알 수 있습니다.
##### name field에 나이와 성별 기혼 등을 의미하는 호칭(Mr, Mrs, Miss)이 있는 것을 알 수 있습니다. 

##### 3.2.1. name column에서 유의미한 호칭만 추출하여 별도의 initial column으로 만들고 기존 데이터에 추가

In [212]:
titanic_df = titanic_df.withColumn('initial', regexp_extract(col('name'), "(\w+)\.", 1))
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|passengerid|survived|pclass|                name|   sex| age|sibsp|parch|          ticket|   fare|cabin|embarked|initial|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|     Mr. Owen Harris|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|     Mr|
|          2|       1|     1|Mrs. John Bradley...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|    Mrs|
|          3|       1|     3|         Miss. Laina|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|   Miss|
|          4|       1|     1|Mrs. Jacques Heat...|female|35.0|    1|    0|          113803|   53.1| C123|       S|    Mrs|
|          5|       0|     3|   Mr. William Henry|  male|35.0|    0|    0|          373450|   8.05|     |       S|     Mr|
|          6|   

##### 3.2.2 생성된 initial feature의 데이터 분포 확인 

In [213]:
titanic_df.groupBy('initial').count().orderBy('count', ascending=[0]).show()

+--------+-----+
| initial|count|
+--------+-----+
|      Mr|  517|
|    Miss|  182|
|     Mrs|  125|
|  Master|   40|
|      Dr|    7|
|     Rev|    6|
|   Major|    2|
|     Col|    2|
|    Mlle|    2|
|Countess|    1|
|     Mme|    1|
|     Don|    1|
|      Ms|    1|
|    Capt|    1|
|    Lady|    1|
|Jonkheer|    1|
|     Sir|    1|
+--------+-----+

##### Mlle나 Mme 같이 Miss를 다르게 표현한 호칭도 확인할 수 있고, Master까지만 유의미하고 나머지는 하나로 그룹화하여 Other로 통합하는 것이 이후 데이터 분석에 효율적일 것으로 생각됩니다.  

In [214]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])
titanic_df.groupBy("initial").count().orderBy("count", ascending=[0]).show()

+-------+-----+
|initial|count|
+-------+-----+
|     Mr|  529|
|   Miss|  186|
|    Mrs|  127|
| Master|   40|
|  Other|    9|
+-------+-----+

##### 최종 정리된 Initial feature의 모습을 확인할 수 있습니다.

#### 3.3. Age

In [215]:
titanic_df.where((col('age').isNull()) | (col('age') == '')).count()

177

##### age에 177개의 빈 값은 적절하게 채워줘야 나중에 모델링할 때 성능을 높일 수 있습니다.

##### 3.3.1. age의 평균값을 확인

In [216]:
# mean('age')의 output column 이름이 mean(age)여서 mean_age로 alias 함 
titanic_df.select(mean('age').alias('mean_age')).show()

+-----------------+
|         mean_age|
+-----------------+
|29.69911764705882|
+-----------------+

##### 3.3.2. 전체 age의 평균은 29.70이나 우리는 initial feature를 알고 각 initial age 평균을 살펴봅니다.

In [217]:
import pyspark.sql.functions as sf

# avg('age')의 output column 이름이 avg(age)여서 avg_age로 alias 함 
titanic_df.groupby('initial').agg(sf.avg('age').alias('avg_age')).orderBy('avg_age', ascending=[1]).show()

+-------+------------------+
|initial|           avg_age|
+-------+------------------+
| Master| 4.574166666666667|
|   Miss|             21.86|
|     Mr| 32.73960880195599|
|    Mrs|35.981818181818184|
|  Other|45.888888888888886|
+-------+------------------+

##### 3.3.3. 각 initial age 평균을 알 수 있으므로 우리는 빈 age 값을 이를 이용하여 좀 더 스마트하게 채워줄 수 있습니다.

In [218]:
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Miss') & (titanic_df['age'].isNull()), 22).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Other') & (titanic_df['age'].isNull()), 46).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Master') & (titanic_df['age'].isNull()), 5).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Mr') & (titanic_df['age'].isNull()), 33).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Mrs') & (titanic_df['age'].isNull()), 36).otherwise(titanic_df['age']))

##### 3.3.4. age의 요약 통계를 다시 확인합니다. null 값없이 891개의 값이 존재하며, 평균도 이전 값을 유지하고 있는 것을 확인할 수 있습니다.

In [219]:
titanic_df.select(col('age')).describe().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|               891|
|   mean|29.841941638608304|
| stddev|13.281524514031313|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+

#### 3.4. Embarked

In [220]:
titanic_df.groupBy('embarked').count().show()

+--------+-----+
|embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  644|
|        |    2|
+--------+-----+

##### 3.4.1. 대부분의 승객이 'S'에 해당하고 missing 값은 단 2개 뿐이므로 missing 값을 'S'로 채워줘도 큰 문제는 없습니다.

In [221]:
titanic_df = titanic_df.na.fill({'embarked' : 'S'})

#### 3.5. Cabin

In [222]:
titanic_df.where((col('cabin').isNull()) | (col('cabin') == '')).count()

687

##### 3.5.1. cabin feacture는 대다수의 값이 빈 값으로 사용하기 어렵다고 판단되므로 drop 하도록 하겠습니다.

In [223]:
titanic_df = titanic_df.drop('cabin')

In [224]:
titanic_df.printSchema()

root
 |-- passengerid: long (nullable = true)
 |-- survived: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: long (nullable = true)
 |-- parch: long (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = false)
 |-- initial: string (nullable = true)

##### Schema에서 cabin feature가 drop 된 것을 확인할 수 있습니다.

#### 3.6. Family Size

##### 3.6.1. sibsp와 parch를 사용해서 family_size feature를 만들어 사용하도록 하겠습니다.

In [225]:
titanic_df = titanic_df.withColumn('family_Size', col('sibsp') + col('parch'))

In [226]:
titanic_df.groupBy('family_size').count().orderBy('count', ascending=[0]).show()

+-----------+-----+
|family_size|count|
+-----------+-----+
|          0|  537|
|          1|  161|
|          2|  102|
|          3|   29|
|          5|   22|
|          4|   15|
|          6|   12|
|         10|    7|
|          7|    6|
+-----------+-----+

##### 3.6.2. family_size를 확인하니 대다수는 0이어서 alone feature를 따로 만들어 모델링에 사용하도록 하겠습니다.
##### alone feature의 기본 값은 0으로 설정합니다.

In [227]:
titanic_df = titanic_df.withColumn('alone', lit(0))

##### family_size가 0이면 alone feature를 1로 설정합니다.

In [228]:
titanic_df = titanic_df.withColumn('alone', when(titanic_df['family_size'] == 0, 1).otherwise(titanic_df['alone']))

#### 3.7. Create Index features

##### 3.7.1. 현재까지 생성된 column들을 살펴봅니다.

In [229]:
titanic_df.columns

['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'embarked', 'initial', 'family_Size', 'alone']

##### 3.7.2. 이 중 sex, embarked, initial feature는 값이 string으로 되어있어 모델링에 사용할 수 없습니다. StringIndexer를 이용해서 number로 변환한 후 별도의 column으로 추가합니다.

In [230]:
indexers = [StringIndexer(inputCol=column, outputCol=column+'_index').fit(titanic_df) for column in ['sex', 'embarked', 'initial']]
pipeline = Pipeline(stages=indexers)
titanic_df = pipeline.fit(titanic_df).transform(titanic_df)
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
|passengerid|survived|pclass|                name|   sex| age|sibsp|parch|          ticket|   fare|embarked|initial|family_Size|alone|sex_index|embarked_index|initial_index|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
|          1|       0|     3|     Mr. Owen Harris|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|     Mr|          1|    0|      0.0|           0.0|          0.0|
|          2|       1|     1|Mrs. John Bradley...|female|38.0|    1|    0|        PC 17599|71.2833|       C|    Mrs|          1|    0|      1.0|           1.0|          2.0|
|          3|       1|     3|         Miss. Laina|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|   Miss|          0|  

##### 3.7.3. 변경된 Schema를 확인합니다.

In [231]:
titanic_df.printSchema()

root
 |-- passengerid: long (nullable = true)
 |-- survived: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: long (nullable = true)
 |-- parch: long (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = false)
 |-- initial: string (nullable = true)
 |-- family_Size: long (nullable = true)
 |-- alone: integer (nullable = false)
 |-- sex_index: double (nullable = true)
 |-- embarked_index: double (nullable = true)
 |-- initial_index: double (nullable = true)

#### 3.8. Drop Columns

##### 3.8.1. 모델링에서 사용하지 않는 column들은 Drop 하도록 합니다.

In [232]:
titanic_df = titanic_df.drop('passengerid', 'name', 'ticket', 'cabin', 'embarked', 'sex', 'initial')

##### 모델링에 사용할 최종 데이터를 확인합니다.

In [233]:
titanic_df.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|survived|pclass| age|sibsp|parch|   fare|family_Size|alone|sex_index|embarked_index|initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|          0.0|


#### 3.9. Put all features into vector

##### 3.9.1. 모델에서 해당 feature들을 사용하기 위해서 VectorAssembler를 이용해서 vector화 합니다.

In [234]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:], outputCol='features')
feature_vector= feature.transform(titanic_df)

##### 3.9.2. features column이 추가된 것을 확인할 수 있습니다.

In [235]:
feature_vector.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|survived|pclass| age|sibsp|parch|   fare|family_Size|alone|sex_index|embarked_index|initial_index|            features|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|[1.0,38.0,1.0,0.0...|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|[3.0,26.0,0.0,0.0...|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|[1.0,35.0,1.0,0.0...|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[3....|
|       0|     3|33.0|    0|    

### 4. Modelling

#### 4.1. 모델 평가를 위해 train 데이터와 test 데이터를 약 80:20 비율로 준비합니다.

In [236]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2], seed=11)

#### 4.2. LogisticRegression

In [237]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='survived', featuresCol='features')
#Training algo
lrModel = lr.fit(trainingData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select('prediction', 'survived', 'features').show()
evaluator = MulticlassClassificationEvaluator(labelCol='survived', predictionCol='prediction', metricName='accuracy')

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       1.0|       0|[1.0,27.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       1.0|       0|[1.0,28.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

#### Evaluating accuracy of LogisticRegression.

In [238]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g" % (lr_accuracy))
print("Test Error of LogisticRegression = %g " % (1.0 - lr_accuracy))

Accuracy of LogisticRegression is = 0.836257
Test Error of LogisticRegression = 0.163743

#### 4.3. DecisionTreeClassifier

In [239]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol='survived', featuresCol='features')
dt_model = dt.fit(trainingData)
dt_prediction = dt_model.transform(testData)
dt_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       0.0|       0|[1.0,27.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,28.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       1.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

#### Evaluating accuracy of DecisionTreeClassifier.

In [240]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy of DecisionTreeClassifier is = %g" % (dt_accuracy))
print("Test Error of DecisionTreeClassifier = %g " % (1.0 - dt_accuracy))

Accuracy of DecisionTreeClassifier is = 0.807018
Test Error of DecisionTreeClassifier = 0.192982

#### 4.4. RandomForestClassifier

In [241]:
from pyspark.ml.classification import RandomForestClassifier
rf = DecisionTreeClassifier(labelCol='survived', featuresCol='features')
rf_model = rf.fit(trainingData)
rf_prediction = rf_model.transform(testData)
rf_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       0.0|       0|[1.0,27.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,28.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       1.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

#### Evaluating accuracy of RandomForestClassifier.

In [242]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("Accuracy of RandomForestClassifier is = %g" % (rf_accuracy))
print("Test Error of RandomForestClassifier  = %g " % (1.0 - rf_accuracy))

Accuracy of RandomForestClassifier is = 0.807018
Test Error of RandomForestClassifier  = 0.192982

#### 4.5. Gradient-boosted tree classifier

In [243]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol='survived', featuresCol='features', maxIter=10)
gbt_model = gbt.fit(trainingData)
gbt_prediction = gbt_model.transform(testData)
gbt_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       1.0|       0|[1.0,27.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       1.0|       0|[1.0,28.0,1.0,0.0...|
|       1.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

#### Evaluate accuracy of Gradient-boosted.

In [244]:
gbt_accuracy = evaluator.evaluate(gbt_prediction)
print("Accuracy of Gradient-boosted tree classifie is = %g" % (gbt_accuracy))
print("Test Error of Gradient-boosted tree classifie %g" % (1.0 - gbt_accuracy))

Accuracy of Gradient-boosted tree classifie is = 0.818713
Test Error of Gradient-boosted tree classifie 0.181287

#### 4.6. NaiveBayes

In [245]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(labelCol='survived', featuresCol='features')
nb_model = nb.fit(trainingData)
nb_prediction = nb_model.transform(testData)
nb_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,19.0,3.0,2.0...|
|       1.0|       0|[1.0,27.0,0.0,2.0...|
|       1.0|       0|(10,[0,1,4,6],[1....|
|       1.0|       0|[1.0,28.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|(10,[0,1,2,4,5],[...|
|       1.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|(10,[0,1,4,6],[2....|
|       1.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       1.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

#### Evaluating accuracy of NaiveBayes.

In [246]:
nb_accuracy = evaluator.evaluate(nb_prediction)
print("Accuracy of NaiveBayes is  = %g" % (nb_accuracy))
print("Test Error of NaiveBayes  = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is  = 0.695906
Test Error of NaiveBayes  = 0.304094

#### 4.7. Support Vector Machine

In [247]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(labelCol='survived', featuresCol='features')
svm_model = svm.fit(trainingData)
svm_prediction = svm_model.transform(testData)
svm_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       0.0|       0|[1.0,27.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,28.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

#### Evaluating the accuracy of Support Vector Machine.

In [248]:
svm_accuracy = evaluator.evaluate(svm_prediction)
print("Accuracy of Support Vector Machine is = %g" % (svm_accuracy))
print("Test Error of Support Vector Machine = %g " % (1.0 - svm_accuracy))

Accuracy of Support Vector Machine is = 0.836257
Test Error of Support Vector Machine = 0.163743

#### 4.8. Result

##### 위 테스트에서는 LogisticRegression과 Support Vector Machine이 83.6257%의 정확도로 가장 좋은 성능을 보여주었습니다.
##### 하지만, 아직 기본적인 테스트만 수행하였으므로 개선할 여지가 많이 남아있습니다.

### 5. Create your own model
![kaggle_leaderboard](images/kaggle_leaderboard.png)
#### 지금까지 다뤘던 기본적인 Feature engineering과 Modeling만으로 83.6%의 비교적 높은 정확도를 얻을 수 있었습니다. 
#### kaggle titanic competition에서 test 데이터에 대해서도 같은 정확도를 유지한다면 전체 10000명 중 상위 229등에 해당하는 높은 수치입니다. 하지만, 아직 개선할 여지가 많이 남아있습니다.
#### * 첫 번째 접근할 수 있는 방법으로는 새로운 feacure를 추가하거나 기존 feature를 모델에서 제외하는 것입니다.
#### * 두 번째 방법은 ML algorithm을 튜닝하는 것으로 아래 페이지를 참고하시면 됩니다.
(https://spark.apache.org/docs/latest/ml-tuning.html) 